### Summary

This notebook presents a typical use of neural network to time series of the EPRI project.

It is strongly suggested to read the Neural Network report provided in the Report folder of the shared box in order to understand the various pre-processing methods we refer to.

Choice was made to leave several possibly conflicting lines of code in the cells; instructions were left to indicate when commenting out lines was desirable.

A lot of ad hoc modifications are possible, especially at the stage of loading the data, as this is the moment where data is preprocessed and the input and output are determined. When moving the code to an implementable .py file **one should rewrite preprocessing into a pipeline, so that raw data could be fed to the machine.**

### How to reproduce results from the Neural Network report:

#### Basic dataset

In order to reproduce the results in the report, follow this procedure:

1) *No subsampling.* For each of the 'choose one of two' mentions in part 2, comment out b). Make sure the second cell of part 2 loads `X_5years` and `Y_5years`.

2) *Downsizing to day-by-day series using the 70th percentile of power for each day.* In part 2.III.iv, edit the tail of the `groupby` line to be `.quantile(q=0.7)`

3) *No other preprocessing.* Comment out part 2.III.iii

4) Make sure that the downsizing (part 2.III.iv) processes the `Power` (as opposed to `Power_norm` or `Power_scaled`).

5) *Model naming.* Part 3.III, rename the model to your convenience.

Training converges to optimal solution within ~500 epochs. Training can be interrupted midway by interrupting the jupyter notebook kernel; best model found during training will still be saved.

#### Soiling, Weather, Soil and Weather datasets

2-year subsampling performing consistently worse than 3-year subsampling and no subsampling, while 4-year subsampling never outperformed 3-year subsampling. Thus we decided not to include these other subsampling methods in this jupyter notebook. If needed it shouldn't be too hard to take example on the 3-year subsampling coded here and adapt it for 2-year or 4-year subsampling.

The various daily aggregations pre-processing can be tested by modifying the tail of the `groupby` line in part 2.III.iv accordingly:
- `.quantile(q=0.7)` for the 70th percentile of power for each day
- `.median()` for the median power for each day
- `.mean()` for the mean power for each day

If you want to use a power clipping function, simply adapt part 2.III.iii by commenting out all but power clipping lines.

#### Warning about optional preprocessing (part 2.III.iii and following)

Most of the preprocessing methods create a new column in the dataframe each time preprocessing is applied. If you want to go through several preprocessing steps (say, scaling, then POA-normalization) make sure you edit the code accordingly so that each step's input corresponds to the output of the previous step (in the example above: scaling takes `df.Power` as input, and produces `df.Power_scaled`; then, normalization takes `df.Power_scaled` as input and produces `df.Power_normalized`). Also, make sure that part 2.III.iv takes as input the final output of the hitherto preprocessing (in the above example, `df.Power_normalized`).

### 1. Imports and functions

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import sklearn
from sklearn import metrics
import math
from tensorflow.keras import initializers
import sys
from tensorflow.keras import backend as K
import datetime

from src.data.make_dataset import remove_clipping_with_flexible_window as remove_clip_flex
from src.data.make_dataset import remove_clipping_with_universal_window as remove_clip_univ

tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
# Converting a decay series into a decay rate.
def extract_yr_degrad(avg_power_decay):
    x = np.array([i/365 for i in range(0,len(avg_power_decay))])
    y = np.array(avg_power_decay)
    return np.polyfit(x,y,1)[0]


# Our personal activation function for the last layer of the neural network
def paf(x):
    return 0.21*K.sigmoid(x)+0.8
tf.keras.utils.get_custom_objects().update({'paf': paf})


# Generating noise to test normalization by irradiance with sensor drift
# Parameters A1 and K1 can be adjusted
def generate_noise(series_length):
    freq = np.fft.rfftfreq(series_length)
    sp = []
    A1 = 0.002
    K1 = 1-1e-3
    for freq_iter in np.nditer(freq):
        if freq_iter == 0:
            sp.append(0)
        else:
            std_iter = A1*freq_iter**(-K1)
            sp_real = np.random.normal(0, std_iter)
            sp_imag = np.random.normal(0, std_iter)
            sp.append(sp_real + 1j*sp_imag)
    sp = np.asarray(sp)
    return np.fft.irfft(sp)+1


# Smoothing a power series by taking a rolling median of the series.
# window is the size of the subarray over which median is taken.
# step is the number of steps by which the median window is shifted
# to compute the next median.
def rolling_median(s, window, step):
    s = pd.Series([0]*(window//2)+list(s)+[0]*(window//2+window%2))
    vert_idx_list = np.arange(0, s.size - window, step)
    hori_idx_list = np.arange(window)
    A, B = np.meshgrid(hori_idx_list, vert_idx_list)
    idx_array = A + B
    x_array = s.values[idx_array]
    idx = s.index[vert_idx_list + int(window/2.)]
    d = np.quantile(x_array,0.5, axis=1)
    return np.array(d)

### 2. Loading and pre-processing data

In [ ]:
# I. Creating datasets of 5-year-long and 3-year-long series
#    X being the input, Y being the target

# Choose one of two (comment out the choice not needed)

# a) 5-year-long input (no subsampling)
X_5years = np.empty((0, 365*5+1), float)
Y_5years = np.empty((0, 365*5+1), float)

# b) 3-year-long input (using subsampling)
X_3years = np.empty((0, 365*3), float)
Y_3years = np.empty((0, 365*3), float)


# II. Choosing the dataset. Comment out datasets you don't need.
# The following assumes that the zip of pkl has been extracted,
# and all the pkl files are in data/raw/synthetic_<name_of_datastage>
datasets_addresses = [
    "synthetic_basic/synthetic_basic",
    "synthetic_soil/synthetic_soil",
    "synthetic_weather/synthetic_weather",
    "synthetic_soil_weather/synthetic_soil_weather",
    "synthetic_soil_weather_locations/synthetic_soil_weather_locations"
]

# III. Loading, preprocessing and adding each time series to input and target arrays

for dataset in datasets_addresses:
    
    files_count = len(
        [f for f in os.listdir("../data/raw/"+dataset.split("/")[0])
         if f.endswith('.pkl')
         and f.startswith(dataset.split("/")[1]+"_")
         and os.path.isfile(os.path.join("../data/raw/"+dataset.split("/")[0], f))]
    )

    for i in range(1, files_count+1):
        print("Loading file #"+str(i)+" from dataset "+dataset.split("/")[0])
            
        # i. Loading a time series
        index = str(i)
        if i < 10:
            index = "00"+str(i)
        elif i <100:
            index = "0"+str(i)
        df = pd.read_pickle("../data/raw/"+dataset+"_"+index+".pkl", "gzip")
        
        # ii. Adding target to the target list, taking for each day the value of the
        #     degradation at the first minute of the day.
        
        # Choose one of two:
        
        # a) For 5-year-long input (no subsampling)
        Y_5years = np.vstack([
            Y_5years,
            np.array(df[df.index.time == datetime.time(0,0)].Degradation)])
    
        # b) For 3-year-long input (subsampling)
        for j in range(0,3):
            Y_3years = np.vstack([
                Y_3years,
                np.array(df[df.index.time == datetime.time(0,0)].Degradation[:365*3])])

        # iii. Optional preprocessing. Comment out any that is not needed.
        
        # Applying power clipping removal
        df["minute_of_day"] = df.index.hour*60+df.index.minute
        df = remove_clip_flex(df)
        
        # Scaling the power signal
        df["Power_scaled"] = (df.Power+1)/df.Power.max()
        
        # Normalizing by irradiance with sensor drift
        drift_noise = generate_noise(len(df) + 1) 
        # generate_noise(n) returns an array of length n-1 if n is odd, so len(df)+1
        # to make sure that an array generated is long enough
        df["Drifting_POA"] = df.POA * drift_noise[:len(df)]
        df = df[df.Drifting_POA > 1.0]
        df["Power_norm"] = df.Power_scaled / df.Drifting_POA
        
        # iv. Downsizing to day-by-day time series.
        #     The function at the tail of the groupby line can be edited to change
        #     the downsizing method.
        
        preprocessed_power = np.array(
            df.groupby(df.index.date, as_index=False).Power_scaled.quantile(q=0.7)
        ).reshape(365*5+1)
        
        # Optional: smoothing the power signal with a rolling median or savgol filter.
        # Usually one of the two lines suffice.
        # Function parameters can be adapted at will.
        
        preprocessed_power = rolling_median(preprocessed_power, 30, 1)
        # preprocessed_power = savgol_filter(preprocessed_power, 30, 2)
        
        # Choose one of two:
        
        # a) For 5-year-long input (no subsampling)
        X_5years = np.vstack([
            X_5years,
            preprocessed_power
        ])
        
        # b) For 3-year-long input (subsampling)
        for j in range(0,3):
            X_3years = np.vstack([
                X_3years,
                preprocessed_power[365*j:365*(j+3)]
            ])

print("All done!")


In [ ]:
# IV. Choose the sample set & target you wanna train on.
#     This step is added in case you want to load
#     BOTH 3-year-long AND 5-year-long input.

X = X_3years
Y = Y_3years

In [ ]:
# V. Optional: check that the data looks like what you expect.

plt.figure(figsize=(15,20))
plt.suptitle("Examples of the loaded data: input")
for i in range(0, 10):
    plt.xlabel("Time steps")
    plt.ylabel("Power")
    plt.subplot(5,2,i+1)
    plt.plot(X[i])
plt.show()

plt.figure()
plt.title("Examples of the loaded data: target")
plt.xlabel("Time steps")
plt.ylabel("Normalized degradation factor")
for i in range(0, 10):
    plt.plot(Y[i])
plt.show()

### 3. Training the neural network

In [ ]:
# I. Dividing the data into training, validation and test sets,
#    in proportion 75%-15%-15%

train_index = int(len(X)*0.7)
valid_index = int(len(X)*0.85)

X_train = X[:train_index]
X_valid = X[train_index:valid_index]
X_test = X[valid_index:]
Y_train = Y[:train_index]
Y_valid = Y[train_index:valid_index]
Y_test = Y[valid_index:]

# II. Shuffling the datasets so that, in the case of 3-year-long data,
#     mini-batch training doesn't get a streak of samples that were
#     all extracted from the same time series.
train_shuffle = np.random.permutation(len(X_train))
valid_shuffle = np.random.permutation(len(X_valid))

X_train = X_train[train_shuffle]
Y_train = Y_train[train_shuffle]
X_valid = X_valid[valid_shuffle]
Y_valid = Y_valid[valid_shuffle]

In [ ]:
# III. Initializing parameters necessary for training, and creating a folder
#      to store the model's log and results
run_index = 0
model_name = "my_nn_model"

current_directory = os.getcwd()
final_directory = os.path.join(current_directory, model_name)
if not os.path.exists(final_directory):
    os.makedirs(final_directory)

In [ ]:
# IV. Training the neural network

# i. To update the name of the model, indicating this is the nth run
run_index = run_index + 1

# ii. Optional: creating and sending log to tensorboard.
run_logdir = os.path.join(os.curdir, model_name+"_log", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

# iii. Saving the best model so far during training.
checkpoint_cb = keras.callbacks.ModelCheckpoint(model_name+"_run_"+str(run_index)+".h5", save_best_only=True)

time_series_length = len(X[0])

# iv. Creating and compiling the model.
model = keras.models.Sequential()

model.add(keras.layers.Input(shape=[time_series_length]))

for i in range(0, 6):
    model.add(
        keras.layers.Dense(
            time_series_length,
            activation="elu",
            kernel_initializer="he_normal"
        ))
    model.add(keras.layers.BatchNormalization())
    
model.add(
    keras.layers.Dense(
        len(Y[0]),
        activation="paf",
        kernel_initializer="he_normal"
    ))

model.compile(loss='mse', optimizer=keras.optimizers.Adam())

# v. Training the model.
print("Training run #"+str(run_index))
history = model.fit(X_train,
                    Y_train,
                    epochs=1000,
                    validation_data=(X_valid, Y_valid),
                    callbacks=[checkpoint_cb, tensorboard_cb])

### 4. Results

In [ ]:
# I. Loading the best parameters and generating a list of predictions.
model = keras.models.load_model(model_name+"_run_"+str(run_index)+".h5")
prediction = model.predict(X_test)

In [ ]:
# II. Visualizing the predictions.
plt.figure(figsize=(10,6))
plt.tight_layout()
plt.suptitle("Normalized decay factor vs time step")
for i in range(0, min(len(prediction), 9)):
    plt.subplot(3,3,i+1)
    plt.ylim(0.94,1.01)
    plt.plot([x for x in prediction[i]])
    plt.plot([x for x in Y_test[i]])
plt.legend(['NN prediction','Actual decay'])
plt.show()

plt.figure(figsize=(10,3))
plt.suptitle("Normalized decay factor vs timestep: predicted (left) vs target (right)")
plt.subplot(1,2,1)
plt.ylim(0.90,1)
for i in range(0,len(prediction)):
    plt.plot(prediction[i])
plt.subplot(1,2,2)
plt.ylim(0.90,1)
for i in range(0,len(Y_test)):
    plt.plot(Y_test[i])
plt.show()

In [ ]:
# IV. Computing the decay rates from the prediction and target, and
#      computing the RMSE between those.
model_pred = []
degradation_rates_test = []
for i in range(0,len(prediction)):
    model_pred.append(extract_yr_degrad(prediction[i]))
    degradation_rates_test.append(extract_yr_degrad(Y_test[i]))
    
plt.figure(figsize=(13,3))
plt.plot(degradation_rates_test[0:50])
plt.plot(model_pred[0:50])
plt.xlabel("Time series sample #")
plt.ylabel("Yearly decay rate")
plt.title("NN prediction vs actual yearly decay rate")
plt.legend(["Actual","NN predicted"])
plt.show()
    
mse = metrics.mean_squared_error(model_pred, degradation_rates_test)
print("Model's degradation RMSE: "+str(round(math.sqrt(mse)*100, 5))+"%/year")
print("Model's R2: "+str(round(sklearn.metrics.r2_score(degradation_rates_test, model_pred), 5)))

In [ ]:
# V. For subsampling: aggregating predictions from the 3-year-long subsamples
#    coming from the same 5-year-long sample, and computing the RMSE.

model_pred_aggr = []
degrad_rate_aggr = []
for i in range(len(model_pred)%3,len(model_pred),3):
    degrad_rate_aggr.append(degradation_rates_test[i])
    model_pred_aggr.append(np.mean(model_pred[i:i+3]))
    
plt.figure(figsize=(13,3))
plt.plot(degrad_rate_aggr)
plt.plot(model_pred_aggr)
plt.xlabel("Time series sample #")
plt.ylabel("Yearly decay rate")
plt.title("NN prediction (via aggregation) vs actual yearly decay rate")
plt.legend(["Actual","NN predicted"])
plt.show()

mse = metrics.mean_squared_error(model_pred_aggr, degrad_rate_aggr)
print("Model's degradation RMSE: "+str(round(math.sqrt(mse)*100, 5))+"%/year")
print("Model's R2: "+str(round(sklearn.metrics.r2_score(degrad_rate_aggr, model_pred_aggr), 5)))

### Best models

This section is left empty for you to copy/paste models and preprocessing that work well. Too often did I want to try a modification and ended up editing and then forgetting about the edits of a promising model.